# Determinants of Economic Growth: 
An Empirical Evaluation of the Australian Economy



aus lit rev - https://treasury.gov.au/sites/default/files/2019-03/01_Economic_geography.pdf
- https://treasury.gov.au/sites/default/files/2019-03/3demo.pdf
file:///C:/Users/kalul/Downloads/Determinants_of_Economic_Growth_An_Empirical_Evalu.pdf

1-1-1995 - 1-1-2022

real gdp - https://fred.stlouisfed.org/series/NGDPRXDCAUA

Gross fixed capital formation as a share of GDP

inflation - https://fred.stlouisfed.org/series/FPCPITOTLZGAUS 

population - https://data.worldbank.org/indicator/SP.POP.TOTL?end=2022&locations=AU&start=1995

hci - https://fred.stlouisfed.org/series/HCIYISAUA066NRUG (2019 only - might omit)

imports and exports - worldbank (to calc trade openness)

## 1. Introduction

Briefly introduce the project, its purpose, and the dataset used.
Provide background information on the Human Development Index (HDI) and its significance.

Data

Annual time series data for the period 1995 - 2022, that was obtained from World Bank Development Indicators [73, 74] was used in this study. The following variables were used: Real GDP (expressed in 2010 U.S. dollars.) at purchaser’s prices; Investment (proxied by gross fixed capital formation as a share of GDP); Inflation (measured by the consumer price index); General government expenditure as a share in GDP (General government expenditure as a share in GDP); Human Capital (representing knowledge spill over effects) was proxied by Human capital index3 , based on years of schooling and returns to education); Demography (proxied by total population); Trade openness (measured by the sum of exports and imports as a proportion of GDP); and Foreign Aid as a proportion to GDP (measured by net official development assistance and official aid received as a share of real GDP). Eviews 9.5 software was used to conduct the empirical analysis.

## 2. Data loading and preprocessing

### 2.1. Loading libraries

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import statsmodels.formula.api as smf

# Specify notebook formatting options
sns.set_style('dark')
sns.color_palette("viridis", as_cmap=True)
sns.set()
sns.set_context('talk')
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', '{:.2f}'.format)

### 2.2. Reading data

In [156]:
real_gdp = pd.read_csv('real_gdp.csv', header=None, skiprows=11, usecols=[0, 1], names=['Year', 'Real GDP'])
real_gdp['Year'] = pd.to_datetime(real_gdp['Year']).dt.year

population = pd.read_csv('population.csv', header=3)
population = population.loc[population['Country Code'] == 'AUS']
population = population.drop(columns=['Indicator Name', 'Indicator Code', 'Country Name', 'Country Code'] + [str(x) for x in range(1960, 1995)]).transpose().reset_index()
population.columns = ['Year', 'Population']

exports = pd.read_csv('exports.csv', header=3)
exports = exports.loc[exports['Country Code'] == 'AUS']
exports = exports.drop(columns=['Indicator Name', 'Indicator Code', 'Country Name', 'Country Code'] + [str(x) for x in range(1960, 1995)]).transpose().reset_index()
exports.columns = ['Year', 'Exports']

imports = pd.read_csv('imports.csv', header=3)
imports = imports.loc[imports['Country Code'] == 'AUS']
imports = imports.drop(columns=['Indicator Name', 'Indicator Code', 'Country Name', 'Country Code'] + [str(x) for x in range(1960, 1995)]).transpose().reset_index()
imports.columns = ['Year', 'Imports']

gov_expenditure = pd.read_csv('gov_expenditure.csv', header=3)
gov_expenditure = gov_expenditure.loc[gov_expenditure['Country Code'] == 'AUS']
gov_expenditure = gov_expenditure.drop(columns=['Indicator Name', 'Indicator Code', 'Country Name', 'Country Code'] + [str(x) for x in range(1960, 1995)]).transpose().reset_index()
gov_expenditure.columns = ['Year', 'Government Expenditure']

gross_fixed_capital_formation = pd.read_csv('gross_fixed_capital_formation.csv', header=3)
gross_fixed_capital_formation = gross_fixed_capital_formation.loc[gross_fixed_capital_formation['Country Code'] == 'AUS']
gross_fixed_capital_formation = gross_fixed_capital_formation.drop(columns=['Indicator Name', 'Indicator Code', 'Country Name', 'Country Code'] + [str(x) for x in range(1960, 1995)]).transpose().reset_index()
gross_fixed_capital_formation.columns = ['Year', 'Gross Fixed Capital Formation']

population['Year'] = population['Year'].astype(int)
real_gdp['Year'] = real_gdp['Year'].astype(int)
inflation['Year'] = inflation['Year'].astype(int)
imports['Year'] = imports['Year'].astype(int)
exports['Year'] = exports['Year'].astype(int)
gross_fixed_capital_formation['Year'] = gross_fixed_capital_formation['Year'].astype(int)
gov_expenditure['Year'] = gov_expenditure['Year'].astype(int)

aus = (population
    .merge(real_gdp, on='Year', how='inner') 
    .merge(inflation, on='Year', how='inner') 
    .merge(imports, on='Year', how='inner') 
    .merge(exports, on='Year', how='inner') 
    .merge(gross_fixed_capital_formation, on='Year', how='inner')
    .merge(gov_expenditure, on='Year', how='inner')
)

aus

,Year,Population,Real GDP,Inflation,Imports,Exports,Gross Fixed Capital Formation,Government Expenditure
0,1995,18004882.00,1060379,4.63,19.83,17.87,25.10,18.64
1,1996,18224767.00,1102405,2.62,19.34,18.89,24.15,18.58
2,1997,18423037.00,1148783,0.22,18.85,19.13,24.04,18.27
3,1998,18607584.00,1206322,0.86,20.44,19.56,25.16,18.26
4,1999,18812264.00,1256449,1.48,20.74,18.29,25.37,18.73
5,2000,19028802.00,1299006,4.46,21.54,19.39,25.94,18.54
6,2001,19274701.00,1334270,4.41,22.07,22.15,23.20,18.54
7,2002,19495210.00,1387780,2.98,20.72,20.73,24.30,18.35
8,2003,19720737.00,1431077,2.73,21.14,19.06,26.05,18.41
9,2004,19932722.00,1487803,2.34,19.85,17.16,26.63,18.30


## 3. Exploratory Data Analysis (EDA)

## 4. ARDL model

$$
\
\Delta LGDP_t = \beta_0 + \beta_1 \Delta LGDP_{t-1} + \beta_2 \Delta LINF_{t-1} + \beta_3 \Delta LPOP_{t-1} + \beta_4 \Delta LGFCF_{t-1} + \beta_5 \Delta LIMP_{t-1} + \beta_6 \Delta LEXP_{t-1} + \beta_7 \Delta LTROP_{t-1} + \beta_8 \Delta LGEX_{t-1} + u_t\
$$


| Variable | Description |
|--------------------|---------------------------------------------------|
| $\Delta LGDP_t$ | First difference of the logarithm of Real GDP |
| $\Delta LGDP_{t-1}$ | First-order lagged difference of the logarithm of Real GDP |
| $\Delta LINF_{t-1}$ | First-order lagged difference of the logarithm of Inflation |
| $\Delta LPOP_{t-1}$ | First-order lagged difference of the logarithm of Population |
| $\Delta LGFCF_{t-1}$ | First-order lagged difference of the logarithm of Gross Fixed Capital Formation |
| $\Delta LIMP_{t-1}$ | First-order lagged difference of the logarithm of Imports |
| $\Delta LEXP_{t-1}$ | First-order lagged difference of the logarithm of Exports |
| $\Delta LTROP_{t-1}$ | First-order lagged difference of the logarithm of Trade Openness |
| $\Delta LGEX_{t-1}$| First-order lagged difference of the logarithm of Government Expenditure |



### Unit root test
Unit roots/stationarity tests were conducted because this is a prime requirement for any co-integration and causality tests. The augmented Dickey-Fuller test was used to test for a unit root in a univariate process in the presence of serial correlation.

ADF test hypotheses: 
* **H0**: The time series is non-stationary. In other words, it has some time-dependent structure and does not have constant variance over time.
* **HA**: The time series is stationary.

The ADF test results were augmented with the Phillips-Perron test. Table 1 presents the results of the unit root tests.

In [ ]:
statsmodels.tsa.stattools.adfuller(aus, maxlag=None, regression='c', autolag='AIC', store=False, regresults=False)[source]¶